# **Feature Engineering — Módulo 2, Notebook 4/4**

---

## Índice

1. [Introdução](#introducao)
2. [Encoding](#encoding)
3. [Scaling](#scaling)
4. [Padronização](#padronizacao)
5. [Imputação de valores ausentes](#imputacao)
6. [Criação de novas features](#criacao-features)

---

<a id='introducao'></a>

A Feature Engineering é o processo de extração, manipulação e transformação de dados brutos, permitindo que esses dados se transformem em variáveis que um modelo de machine learning pode usar para alcançar um objetivo desejado. A engenharia de recursos é um desafio porque envolve uma combinação de análise de dados, conhecimento do domínio de negócios e certa intuição.

Pense que dados, por si só, podem não significar muita coisa. Esse problema se torna ainda mais agravante nos dias atuais quando pensamos na quantidade massiva de dados gerados na era do big data. Mais do que nunca, temos um mar de dados a nossa disposição. Porém, devemos pensar, como tirar valor disso? Podemos pensar no processo de **feature engineering** de forma abrangente, como a maneira de transformar **dados** em **informação**.

Uma analogia é imaginar o trabalho de um minerador de ouro. Ele extrai rochas brutas (dados) e precisa processá-las para encontrar o ouro (informação útil). Do mesmo modo, a Feature Engineering extrai informações valiosas dos dados e traduz em uma linguagem que o modelo de machine learning consiga ler.

<p align=center>
<img src="https://www.kdnuggets.com/wp-content/uploads/Feature-Engineering.png" width=600></img>
</p>

Existem algumas recomendações para uma feature engineering que sempre devemos seguir e iremos abordar elas nos próximos tópicos. Porém, é necessário termos em mente que boa parte do processo de feature engineering vai variar em cada situação específica e boa parte dela vem de termos criatividade e expertise para olharmos para nossos dados e imaginar como podemos extrair valor deles.

---

<a id='encoding'></a>

### **Encoding**

De longe, a maneira mais comum de se representar variáveis categóricas é utilizando encoding. Imagine que uma de suas features é uma coluna de nível educacional com valores como *Fundamental*, *Ensino Médio*, *Superior* e *Pós-Graduação*. Lembre-se que os modelos de machine learning são algoritmos matemáticos/estatísticos. Dessa forma, não podemos utilizar variáveis textuais diretamente no modelo; precisamos traduzir isso em linguagem matemática.


#### **One-hot encoding**
O método mais conhecido e utilizado é o **one-hot encoding** que cria uma coluna binária para cada categoria. Essas colunas também são chamadas de variáveis dummies.

No nosso exemplo, teríamos uma coluna "Fundamental" que receberia valor 1 para cada observação com este nível educacional e 0 caso contrário, uma coluna "Ensino Médio" com valor 1 para cada observação com esse nível educacional e 0 caso contrário, e assim por diante.

📝 **Observação**: Devemos ter cuidado ao aplicar o one-hot encoding para não incorrermos no problema de multicolinearidade. Esse problema acontece quando uma coluna pode ser perfeitamente prevista a partir das outras. Isso cria redundância nos dados e pode atrapalhar alguns modelos estatísticos, como regressão linear, ao tornar os cálculos instáveis.

No nosso exemplo, a última coluna (Pós-Graduação) pode ser perfeitamente inferida das outras três, pois: Pós-Graduação = 1 − (Fundamental + Ensino Médio + Superior). Para contornar esse problema, basta excluirmos uma coluna ao fazer o one-hot encoding.

#### **Label encoding**

Outro método bastante utilizado é o label encoding. De maneira simples, ele converte os valores de uma variável categórica em números inteiros. No nosso exemplo, "Fundamental" seria convertido para 0, "Ensino Médio" = 1, "Superior" = 2 e "Pós-Graduação" = 3.

📝 **Observação**: Devemos ter cuidado ao utilizar o label encoding porque os modelos podem interpretar essa transformação como uma relação ordinal. Por exemplo, o modelo pode entender que "Pós-Graduação" (3) é três vezes mais importante do que "Fundamental" (0), o que pode levar a inferências incorretas.

#### **Prática**

Para ilustrar a aplicação de encoding em python, iremos importar um novo dataset que possui variáveis categoricas.

Este dataframe contém dados de tripulantes do titanic.

In [ ]:
from sklearn.datasets import fetch_openml
import pandas as pd
import numpy as np

# Carrega o dataset
titanic = fetch_openml('titanic', version=1, as_frame=True)
# Transforma em um DataFrame
df_titanic = pd.DataFrame(data=titanic.data, columns=titanic.feature_names)

# Adiciona a coluna de classes
df_titanic['survived'] = titanic.target

# Tradução das colunas para o portugues
df_titanic = df_titanic.rename(columns={'pclass': 'Classe',
                                        'name': 'Nome',
                                        'sex': 'Sexo',
                                        'age': 'Idade',
                                        'sibsp': 'Irmãos/Cônjuges',
                                        'parch': 'Pais/Filhos',
                                        'ticket': 'Bilhete',
                                        'fare': 'Tarifa',
                                        'cabin': 'Cabine',
                                        'embarked': 'Embarque',
                                        'boat': 'Bote',
                                        'body': 'Corpo',
                                        'home.dest': 'Destino',
                                        'survived': 'Sobreviveu'
                                        })

# E exibe as 5 primeiras linhas
df_titanic.head(3)

In [ ]:
# Importa a função de one-hot encoding do sklearn
from sklearn.preprocessing import OneHotEncoder

# Instancia o OneHotEncoder
encoder = OneHotEncoder(drop='first')

# Aplica o one-hot encoding nas colunas categóricas
categorical_cols = ['Cabine']
X_titanic = df_titanic.drop(columns=['Sobreviveu'])
y_titanic = df_titanic['Sobreviveu']
X_titanic_encoded = encoder.fit_transform(X_titanic[categorical_cols]).toarray()
X_titanic_encoded_df = pd.DataFrame(X_titanic_encoded, columns=encoder.get_feature_names_out(categorical_cols))
X_titanic_encoded_df = pd.concat([X_titanic.drop(columns=categorical_cols).reset_index(drop=True), X_titanic_encoded_df], axis=1)

# E exibe as 5 primeiras linhas
X_titanic_encoded_df.head(3)

Perceba a quantidade de colunas resultante. Talvez o one hot encoding não seja a melhor maneira de tratar a variável categorica neste caso.

In [ ]:
# Importa a função de label encoding do sklearn
from sklearn.preprocessing import LabelEncoder

# Instancia o LabelEncoder
label_encoder = LabelEncoder()

# Aplica o label encoding nas colunas categóricas
categorical_cols = ['Cabine']

X_titanic = df_titanic.drop(columns=['Sobreviveu'])
y_titanic = df_titanic['Sobreviveu']
X_titanic[categorical_cols] = X_titanic[categorical_cols].apply(label_encoder.fit_transform)

# E exibe as 5 primeiras linhas
X_titanic.head(3)

Perceba que agora, a variável "Cabine" não é mais categorica, e sim númerica.

### **Scaling**

<a id='scaling'></a>

Scaling é o processo de transformar os valores númericos de uma base de dados para que eles fiquem dentro de uma mesma escala. Pense em uma base de dados em que temos idade e salário de várias pessoas. A variável idade deverá ter um range com algo em torno de 0 a 100, enquanto o salário pode ir de 0 a R$ 1.000.000,00 por exemplo. Muitos algoritmos de machine learning são sensíveis a escalas diferentes entre os dados.

In [ ]:
# Importa a função de min max scaling do sklearn
from sklearn.preprocessing import MinMaxScaler

# Instancia o MinMaxScaler
scaler = MinMaxScaler()

# Aplica o min max scaling na coluna idade
X_titanic = df_titanic.drop(columns=['Sobreviveu'])
y_titanic = df_titanic['Sobreviveu']
X_titanic['Idade'] = scaler.fit_transform(X_titanic[['Idade']])
# E exibe as 5 primeiras linhas
X_titanic.head(3)

### **Padronização**

<a id='padronizacao'></a>

Padronização é o processo de transformar a **distribuição** de uma variável de modo que ela tenha média 0 e desvio padrão 1. Esse é um método que pode ser interessante de usar quando os dados possuem uma distribuição normal ou quando há outliers, pois a média e o desvio padrão são menos afetados que os valores extremos no Min-Max Scaling.

In [ ]:
# Importa a função de normalização do sklearn
from sklearn.preprocessing import StandardScaler

# Instancia o StandardScaler
scaler = StandardScaler()

# Aplica a normalização na coluna idade
X_titanic = df_titanic.drop(columns=['Sobreviveu'])
y_titanic = df_titanic['Sobreviveu']
X_titanic['Idade'] = scaler.fit_transform(X_titanic[['Idade']])

# E exibe as 5 primeiras linhas
X_titanic.head(3)

### **Imputação de valores ausentes**

<a id='imputacao'></a>

Imputação de valores ausentes é preencher lacunas quando certos dados não foram registrados. Ex.: um sensor meteorológico que falhou em medir a temperatura em algumas horas; um questionário online com a cidade deixada em branco; um log de e-commerce sem o tempo de permanência de certos usuários. Para lidar com isso, pode-se usar média/mediana (numéricos), moda (categóricos), forward/backward fill em séries temporais, KNN ou modelos de imputação multivariada. Faça a imputação apenas no conjunto de treino (ou dentro de cada fold) para evitar vazamento de informação e valide o impacto nas métricas do modelo.

In [ ]:
# Exemplo fictício de dados com valores ausentes
df_clientes = pd.DataFrame({
    'Idade': [25, np.nan, 42, 31, np.nan],
    'Salario': [3000.0, 4500.0, np.nan, 5200.0, 4100.0],
    'Cidade': ['A', 'B', None, 'A', None]
})

# Separa features (X) e alvo (y) apenas como exemplo (suponha que 'Cidade' não seja alvo)
X = df_clientes.copy()

# Imputação:
# - Numéricos: média para Idade, mediana para Salario
# - Categórico: moda para Cidade
X['Idade']   = X['Idade'].fillna(X['Idade'].mean())
X['Salario'] = X['Salario'].fillna(X['Salario'].median())
X['Cidade']  = X['Cidade'].fillna(X['Cidade'].mode().iloc[0])

print("Valores ausentes após a imputação:")
print(X.isnull().sum())

# Visualiza as 3 primeiras linhas
X.head()

##### 💡 Alternativas possíveis:

- Imputação por moda 

- _KNN Imputer_

- _Iterative Imputer_

### **Criação de novas features**

<a id='criacao-features'></a>

A criação de novas features nada mais é do que transformar ou combinar variáveis para revelar padrões que o modelo sozinho talvez não capture. Exemplos comuns incluem: transformações (log, raiz, padronização), interações (produto/razão entre variáveis), agregações temporais (médias móveis), extração de componentes de data (mês, dia da semana) e codificação de categóricas. Sempre crie features usando apenas o treino (ou dentro de cada fold) para evitar vazamento de informação.

In [ ]:
# Exemplo fictício
df = pd.DataFrame({
    'Idade':  [22, 35, 41, 29, 33, 26],
    'Salario':[2500, 5200, 6800, 3900, 4500, 3100],
    'Cidade': ['A', 'B', 'A', 'C', 'B', 'A'],
    'Data':   pd.to_datetime(['2025-01-01','2025-01-02','2025-01-03','2025-01-04','2025-01-05','2025-01-06'])
})

# Trabalhe numa cópia
X = df.copy()

# 1) Transformações
X['log_Salario'] = np.log1p(X['Salario'])              # log(1 + Salario)
X['Salario_por_Idade'] = X['Salario'] / (X['Idade']+1) # razão simples (evita div. por zero)

# 2) Interações simples
X['Idade_x_logSal'] = X['Idade'] * X['log_Salario']

# 3) Extração de componentes temporais
X['Mes'] = X['Data'].dt.month
X['DiaSemana'] = X['Data'].dt.dayofweek  # 0 = segunda

# 4) Agregações temporais (ex.: média móvel de 3 dias do salário)
X = X.sort_values('Data')
X['Salario_MM3'] = X['Salario'].rolling(window=3, min_periods=1).mean()

# 5) Codificação de categóricas (one-hot)
X = pd.get_dummies(X, columns=['Cidade'], drop_first=True)

print("Colunas após feature engineering:")
print(list(X.columns))

print("\nPrévia dos dados com novas features:")
print(X.head(10))

Esses são alguns métodos usuais de feature engineering, mas não são todos. Conforme ressaltado, a depender do problema em questão você deverá ter um pingo de criatividade para pensar formas diferentes de extrair informação contida nos dados. Por exemplo, muitas vezes você pode criar novas variáveis fazendo interações de variáveis em sua base.

Sempre dedique um tempo especial a parte de feature engineering quando estiver desenvolvendo um modelo de machine learning, ela pode ser o diferencial para alcançar um resultado satisfatório!

---

<-- [**Anterior: Métricas de Regressão**](03_metricas_regressao.ipynb) | [**Próximo: Módulo 3 — Classificação**](../03_Classificacao/01_knn.ipynb) -->